### notebook 경로 변경

In [1]:
import os
import sys
print(f"현재 노트북 경로: {os.getcwd()}")
# os.chdir("원하는 노트북 경로 설정...")
print(f"파이썬 라이브러리 인식 경로: {sys.path}")

현재 노트북 경로: /Users/admin/workspace/python/DE
파이썬 라이브러리 인식 경로: ['/Users/admin/workspace/python/DE', '/Users/admin/.pyenv/versions/3.10.12/lib/python310.zip', '/Users/admin/.pyenv/versions/3.10.12/lib/python3.10', '/Users/admin/.pyenv/versions/3.10.12/lib/python3.10/lib-dynload', '', '/Users/admin/workspace/python/DE/.venv/lib/python3.10/site-packages']


### 라이브러리 임포트

In [46]:
import pandas as pd
from missions.W1.M3.utils.transform.transformer import numstr2num

### 데이터 로드

In [47]:
df = pd.read_json("missions/W1/M3/data/Countries_by_GDP.json")
df

,country,gdp,year,type
0,United States,"30,337,162",2025.0,IMF
1,China,"19,534,894",2025.0,IMF
2,Germany,"4,921,563",2025.0,IMF
3,Japan,"4,389,326",2025.0,IMF
4,India,"4,271,922",2025.0,IMF
...,...,...,...,...
204,Kiribati,311,2024.0,IMF
205,Palau,308,2024.0,IMF
206,Marshall Islands,305,2024.0,IMF
207,Nauru,161,2024.0,IMF


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  209 non-null    object 
 1   gdp      194 non-null    object 
 2   year     194 non-null    float64
 3   type     209 non-null    object 
dtypes: float64(1), object(3)
memory usage: 6.7+ KB


### 분석을 위한 Transform (string -> float)

In [49]:
df['gdp'] = df['gdp'].apply(numstr2num)
df

,country,gdp,year,type
0,United States,30337162.0,2025.0,IMF
1,China,19534894.0,2025.0,IMF
2,Germany,4921563.0,2025.0,IMF
3,Japan,4389326.0,2025.0,IMF
4,India,4271922.0,2025.0,IMF
...,...,...,...,...
204,Kiribati,311.0,2024.0,IMF
205,Palau,308.0,2024.0,IMF
206,Marshall Islands,305.0,2024.0,IMF
207,Nauru,161.0,2024.0,IMF


In [50]:
df['gdp'] = (df['gdp'] / 1000).round(2)
df

,country,gdp,year,type
0,United States,30337.16,2025.0,IMF
1,China,19534.89,2025.0,IMF
2,Germany,4921.56,2025.0,IMF
3,Japan,4389.33,2025.0,IMF
4,India,4271.92,2025.0,IMF
...,...,...,...,...
204,Kiribati,0.31,2024.0,IMF
205,Palau,0.31,2024.0,IMF
206,Marshall Islands,0.30,2024.0,IMF
207,Nauru,0.16,2024.0,IMF


### Region 추가 

In [51]:
from missions.W1.M3.utils.transform.transformer import country2region
df['region'] = df['country'].map(country2region)

### gdp 순 정렬

In [52]:
df = df.sort_values(by='gdp', ascending=False)

### gdp가 1000억 달러 이상인 국가만 추출

In [53]:
over100b = df[df['gdp'] >= 100]
over100b

,country,gdp,year,type,region
0,United States,30337.16,2025.0,IMF,North America
1,China,19534.89,2025.0,IMF,East Asia
2,Germany,4921.56,2025.0,IMF,Western Europe
3,Japan,4389.33,2025.0,IMF,East Asia
4,India,4271.92,2025.0,IMF,South Asia
...,...,...,...,...,...
68,Uzbekistan,112.65,2024.0,IMF,Central Asia
69,Guatemala,112.37,2024.0,IMF,Latin America
70,Oman,109.99,2024.0,IMF,Middle East
71,Bulgaria,108.42,2024.0,IMF,Eastern Europe


### Region별 Top 5 gdp 평균

In [54]:
# 단순 region 별 top 5 평균값
# region_top5_mean = df.groupby('region').apply(lambda x: x['gdp'].nlargest(5).mean())
# region_top5_mean

# 각 그룹의 top 5 국가들과 값을 먼저 구한 후
top_5_by_group = df.sort_values('gdp', ascending=False).groupby('region').head(5)

# 국가 이름을 리스트로 모으고
countries_by_group = top_5_by_group.groupby('region')['country'].agg(list).reset_index()

# 평균값 계산
means_by_group = top_5_by_group.groupby('region')['gdp'].mean()

# 결과 merge
result = pd.merge(countries_by_group, means_by_group, on='region')
result = result.sort_values('gdp', ascending=False)
result['gdp'] = result['gdp'].round(2)
result


,region,country,gdp
6,North America,"[United States, Canada, Mexico, Aruba, Bermuda]",8622.34
1,East Asia,"[China, Japan, South Korea, Taiwan, Hong Kong]",5421.57
11,Western Europe,"[Germany, United Kingdom, France, Italy, Spain]",3244.49
2,Eastern Europe,"[Russia, Turkey, Poland, Romania, Czech Republic]",1066.60
8,South Asia,"[India, Bangladesh, Pakistan, Sri Lanka, Nepal]",1043.38
3,Latin America,"[Brazil, Argentina, Colombia, Chile, Peru]",791.57
9,Southeast Asia,"[Indonesia, Singapore, Thailand, Malaysia, Phi...",711.89
4,Middle East,"[Saudi Arabia, United Arab Emirates, Israel, I...",598.13
7,Oceania,"[Australia, New Zealand, Papua New Guinea, Fij...",436.66
10,Sub-Saharan Africa,"[South Africa, Nigeria, Ethiopia, Kenya, Angola]",197.53
